### Trynig to understand wfdb 
docs link [https://wfdb.readthedocs.io/en/latest/wfdb.html](https://wfdb.readthedocs.io/en/latest/wfdb.html)

In [2]:
import wfdb

specifing the mit-bih dir
> The name of the WFDB record to be read (without any file extensions). If the argument contains any path delimiter characters, the argument will be interpreted as PATH/baserecord and the data files will be searched for in the local path.

In [4]:
def record_path(record_name: str) -> str:
    mitbih_base_path = 'mit-bih'
    return f'{mitbih_base_path}/{record_name}'

In [4]:
record=wfdb.rdrecord(record_path('100'))
len(record.p_signal) == record.sig_len

True

In [5]:
record.p_signal[:10]

array([[-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       [-0.12 , -0.08 ],
       [-0.135, -0.08 ]])

In [6]:
record_annon=wfdb.rdann(record_path('100'),'atr',return_label_elements=['symbol', 'label_store'])
record_annon.symbol[:10], record_annon.label_store[:10]

(['+', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'A', 'N'],
 array([28,  1,  1,  1,  1,  1,  1,  1,  8,  1]))

In [7]:
record_annon.sample[-10:], len(record.p_signal)

(array([647672, 647934, 648203, 648477, 648733, 648978, 649232, 649484,
        649734, 649991]),
 650000)

In [8]:
record_annon.sample[-1] + 45 <= len(record.p_signal)  # last annotation + 45 should be less than signal length

np.False_

In [9]:
len(record_annon.sample)

2274

In [10]:
record.fs

360

In [11]:
import pandas as pd
import numpy as np

In [12]:
def peak_to_dict(signal_lead: np.ndarray, r_index: int, rr_interval: np.int16, label: str) -> dict:
    peak_dict = {}
    for i, val in enumerate(signal_lead[r_index-45:r_index+46]):
        peak_dict[f'samp_{i}'] = val
    peak_dict['rr_interval'] = rr_interval
    peak_dict['label'] = label
    
    return peak_dict

In [13]:
rr_samples = np.diff(record_annon.sample)
rr_intervals = rr_samples / record.fs * 1000  # in milliseconds

rr_intervals

array([163.88888889, 813.88888889, 811.11111111, ..., 700.        ,
       694.44444444, 713.88888889], shape=(2273,))

In [14]:
def get_rr_intervals(annon_samples: np.ndarray, fs: int) -> np.ndarray:
    rr_samples = np.diff(annon_samples)
    rr_intervals = rr_samples / fs * 1000  # in milliseconds
    return rr_intervals

In [15]:
get_rr_intervals(record_annon.sample, record.fs)

array([163.88888889, 813.88888889, 811.11111111, ..., 700.        ,
       694.44444444, 713.88888889], shape=(2273,))

In [16]:
def is_beat_annotation(symbol: str) -> bool:
    beat_symbols = {'N', 'L', 'R', 'B', 'A', 'a', 'J', 'S', 'V', 'r', 'F', 'e', 'j', 'n', 'E', '/', 'f', 'Q', '?'}
    return symbol in beat_symbols

In [17]:
def get_record_peaks(record_name: str) -> pd.DataFrame:
    record = wfdb.rdrecord(record_path(record_name))
    record_annon = wfdb.rdann(record_path(record_name), 'atr', return_label_elements=['symbol', 'label_store'])
    
    rr_intervals = get_rr_intervals(record_annon.sample, record.fs)
    
    peaks_data = []
    for i in range(1, len(record_annon.sample)-1):
        if is_beat_annotation(record_annon.symbol[i]):
            r_index = record_annon.sample[i]
            rr_interval = rr_intervals[i-1]  # rr_interval corresponds to the interval before the current beat
            label = record_annon.symbol[i]
            peak_dict = peak_to_dict(record.p_signal[:, 0], r_index, rr_interval, label)
            peaks_data.append(peak_dict)
    
    return pd.DataFrame(peaks_data)

In [18]:
record_annon.sample[-1]

np.int64(649991)

In [19]:
def shoud_skip_last_peak(last_r_signal_index, last_peak_index) -> bool:
    return last_r_signal_index + 45 > last_peak_index

In [20]:
shoud_skip_last_peak(record_annon.sample[-1], len(record.p_signal))

np.True_

In [21]:
len(record_annon.sample) - ( 1 if shoud_skip_last_peak(45, 89) else 0)

2273

In [22]:
len(record_annon.sample)

2274

In [ ]:
def write_recording_to_txt_file(rec_name):
    record=wfdb.rdrecord(record_path(rec_name))
    raw_data = record.p_signal[:, 0]

    file = open(f'rec{rec_name}_raw', 'w+')
    file.writelines(list(map(lambda x: str(x)+"\n", list(raw_data))))